<a href="https://colab.research.google.com/github/FNSY96/TerrorismEventsPrediction/blob/master/Graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python Imports**
PLEASE PUT ALL LIBRARY IMPORTS IN THIS CELL **ONLY**

In [1]:
import pandas as pd
import csv
import glob
import os
from pandas import ExcelWriter
from pandas import ExcelFile
import io
print("imports run")

imports run


In [2]:
!pip3 install pandas

# Data Set Imports

In [3]:
!pip install -U -q PyDrive
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

    100% |████████████████████████████████| 993kB 751kB/s 


In [4]:
# drive = GoogleDrive(gauth)
drive.mount("/content/drive", force_remount=True)
datadir  = 'drive/My Drive/GraduationProject/Data'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


###**GTD**

In [0]:
# !ls drive/'My Drive'/GraduationProject/Data/gtd.csv

# Functions

In [0]:
def remove_NaN(data_set, attributes_NaN):
  data_set = data_set.fillna(data_set.mean())
  for i in range(len(attributes_NaN)):
      attribute = attributes_NaN[i]
      mode =  data_set[attribute].mode()[0]
      data_set[attribute] = data_set.groupby(attribute)[attribute].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
      data_set[attribute]= data_set[attribute].fillna(data_set[attribute].value_counts().idxmax())
  return data_set

# Pandas Data Frames
Useful link : https://youtu.be/2AFGPdNn4FM

## **GTD Data Frame**

In [7]:
gtd_df = pd.read_csv(os.path.join(datadir, 'gtd.csv'),encoding='latin-1')
# print(gtd_df.columns.values)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
gtd_filtered_df = gtd_df[['country_txt','region_txt','city','latitude', 'longitude', 'target1', 'iyear']]
# print(gtd_filtered_df)
# print(gtd_filtered_df.isna().sum())

## GTD Removed NaN

In [0]:
# DO NOT REMOVE THIS ONE WE COMMENTED IT TEMORARLY
# clean NaN
# gtd_filtered_df = gtd_filtered_df.fillna(gtd_filtered_df.mean())
# attributes_NaN = ['country_txt', 'city', 'target']
# gtd_filtered_df = remove_NaN(gtd_filtered_df, attributes_NaN)
# print(gtd_filtered_df)

## **GECON Data Frame**

In [10]:
gecon_df = pd.read_csv(os.path.join(datadir, 'Gecon40_post_final_kh.csv'),encoding='latin-1')
# print(gecon_df.columns.values)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,19,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
gecon_filtered_df = gecon_df[['AREA', 'COUNTRY', 'DIS_LAKE','DIS_MAJOR_RIVER']]

# print(gecon_filtered_df.isna().sum())

# print(gecon_filtered_df)

In [0]:
# clean NaN
# gecon_filtered_df = gecon_filtered_df.fillna(gecon_filtered_df.mean())
gecon_filtered_df = remove_NaN(gecon_filtered_df, [])

# print(gecon_filtered_df.isna().sum())

# print(gecon_filtered_df)

## **GeoEPR Data Frame**

In [0]:
geo_df = pd.read_csv(os.path.join(datadir, 'GeoEPR-2018.1.csv'),encoding='latin-1')
# print(geo_df.columns.values)
# # statename, group
# print(geo_df)

In [0]:
geo_filtered_df = geo_df[['statename','group']]
# print(geo_filtered_df)
# print(geo_filtered_df.isna().sum())

In [0]:
# clean NaN
# geo_filtered_df = geo_filtered_df.fillna(geo_filtered_df.mean())
#attributes_NaN = ['statename', 'group', 'type']
#geo_filtered_df = remove_NaN(geo_filtered_df, attributes_NaN)

#print(geo_filtered_df)

## **Happiness World Report Data Frame**

In [0]:
happy_df = pd.read_csv(os.path.join(datadir, 'Happiness2017.csv'),encoding='latin-1')
# print(happy_df.columns.values)

In [0]:

happy_filtered_df = happy_df[['Country','Happiness.Rank','Happiness.Score']]
# print(happy_filtered_df)

In [0]:
# clean NaN
happy_filtered_df = happy_filtered_df.fillna(happy_filtered_df.mean())
happy_filtered_df = remove_NaN(happy_filtered_df, ['Country'])
happy_filtered_df = happy_filtered_df.rename(index=str, columns={"Country": "country_txt"})
# print(happy_filtered_df.columns.values)

In [0]:
# d = {'one': pd.Series([1., 3.], index=['a', 'c']), 'two': pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd']), 'three': pd.Series(["A", "B", "A"], index=['a', 'c', 'd'])}
# df = pd.DataFrame(d)
# print(df)
# # print(df.isna().sum())
# df = df.fillna(df.mean())
# mode =  df['three'].mode()[0]
# df['three'] = df.groupby('three')['three'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
# df['three']= df['three'].fillna(df['three'].value_counts().idxmax())
# print(df)

## MERGED DATA

In [0]:
#result = pd.concat([gtd_filtered_df, happy_filtered_df],gtd_filtered_df["country_txt"]=happy_filtered_df["Country"])
#'Happiness.Rank' 'Happiness.Score'

gtd_merged_happiness = pd.merge(gtd_filtered_df,happy_filtered_df,on='country_txt')
# print(gtd_merged_happiness['region_txt'].drop_duplicates())

In [21]:
mena = gtd_merged_happiness.loc[gtd_merged_happiness['region_txt'] == 'Middle East & North Africa']
print(mena.shape)

(48233, 9)
